# Podstawy Sztucznej Inteligencji 2020/2021


Prosze uzupelnic kod tam gdzie znajduje napis `YOUR CODE HERE` lub 'YOUR ANSWER HERE'.

Warto zresetowac 'kernel' i sprawdzic czy caly notatnik uruchamiany od poczatku nie daje bledow.

---

## Metoda Walidacji Krzyżowej

Jest to jedna z metod estymacji (oszacowania) błędu klasyfikatora. W metodzie tej wykonywany jest podział próby na `v` podzbiorów. `v-1` z nich używamy do uczenia, jeden do
sprawdzenia. Procedurę powtarzamy `v` razy.

Metoda ta jest szczególnie istotna, podczas poszukiwania optymalnych parametrów klasyfikatora.

In [75]:
%matplotlib inline 
import matplotlib.pyplot as plt
import numpy as np

In [76]:
from __future__ import print_function
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn import datasets
from sklearn.svm import SVC
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

Celem zestawu jest znalezienie optymalnego klasyfikator dla danych irys. W tym celu przeszukamy przestrzeń hiperparametrów dla klasyfikatorów kNN i SVM.

### Część 1

Wczytaj dane irys, 
- podziel je na zbiór uczący i testowy w proporcji 7:3 oraz  dokonaj ich standaryzacji metodą `preprocessing.StandardScaler`. Podczas dzielenia ustaw `random_state` na 1234. W metodzie `fit`  obiektu klasy `preprocessing.StandardScaler` uwzględij tylko dane treningowe, natomiast dokonaj standaryzacji zarówno danych testowych, jak i treningowych.

- stwórz zbiór składający się ze wszystkich dostępnych danych i wykonaj dla niego standaryzację. 

In [63]:
standard_train = None
standard_test = None
train_targets = None
test_targets = None

standard_all = None
all_targets = None


#-----


iris_data = datasets.load_iris()
x = iris_data.data

y = iris_data.target
#print(type(y))
#y = np.array(y)

train, test, train_targets, test_targets = train_test_split(x, y, test_size=0.3, random_state=1234)

scaler = preprocessing.StandardScaler(with_std=True).fit(train)

standard_train = scaler.transform(train)
standard_test = scaler.transform(test)

scaler_all = preprocessing.StandardScaler(with_std=True).fit(x)
standard_all = scaler_all.transform(x)

all_targets = y


In [64]:
np.testing.assert_array_almost_equal(np.mean(standard_train, axis=0), [0,0,0,0])
np.testing.assert_array_almost_equal(np.std(standard_train, axis=0), [1,1,1,1])

In [65]:
np.testing.assert_array_almost_equal(np.mean(standard_all, axis=0), [0,0,0,0])
np.testing.assert_array_almost_equal(np.std(standard_all, axis=0), [1,1,1,1])

### Część 2

Dokonaj klasyfikacji z wykorzystaniem klasyfika kNN dla `k = 1`. Oszacuj błąd klasyfikacji z wykorzystaniem metody walidacji krzyżowej. W tym celu wykorzystaj zbiór `standard_all`. Użyj metody `cross_val_score` z biblioteki `sklearn`. Porownaj wynik walidacji krzyżowej z oszacowaniem dokladności z wykorzystaniem danych testowych. Jakie są czasy obu operacji?

In [66]:
clf = KNeighborsClassifier(n_neighbors=1, n_jobs=1).fit(standard_train, train_targets) # k, n_jobs
print(clf.score(standard_test, test_targets))

all_clf = cross_val_score(clf, standard_all, all_targets, cv=5, n_jobs=1)
print(all_clf.std())
print("Dokładność CV: %0.2f (+/- %0.2f)" % (all_clf.mean(), all_clf.std() * 2))

0.9555555555555556
0.03399346342395189
Dokładność CV: 0.95 (+/- 0.07)


### Część 3

Samodzielnie (nie korzystając z funkcji bibliotecznej)wykonaj procedurę walidacji krzyżowej. W pierwszym kroku podziel zbiór danych na 10 części. Wykorzystaj do tego metodę `KFold`. Ustaw `random_state` na 123. 

Następnie metodą split stwórz 10 podziałów na dane testowe i treningowe Dla każdego podziłu (w pętli) trenuj klasyfikator na danych treningowych i oszacuj błęd na danych testowych. Otrzymane wyniki zapisz do tablicy. Po wyjściu z pętli wyznacz średni błąd.

In [98]:
kf = KFold(n_splits=10, random_state=123)
scores = []

for train_index, test_index in kf.split(standard_all):
    #print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = standard_all[train_index], standard_all[test_index]
    Y_train, Y_test = y[train_index], y[test_index]
    
    clf = KNeighborsClassifier(n_neighbors=1, n_jobs=4).fit(X_train, Y_train)
    print(clf.score(X_test, Y_test))
    scores.append(clf.score(X_test, Y_test))

print ("Poszczególne wyniki: ", scores)
print ("Błąd wyznaczony procedurą walidacji krzyżowej: ", np.array(scores).mean())

1.0
1.0
1.0
0.9333333333333333
0.8
0.8666666666666667
1.0
0.8666666666666667
0.8666666666666667
1.0
Poszczególne wyniki:  [1.0, 1.0, 1.0, 0.9333333333333333, 0.8, 0.8666666666666667, 1.0, 0.8666666666666667, 0.8666666666666667, 1.0]
Błąd wyznaczony procedurą walidacji krzyżowej:  0.9333333333333333


In [99]:
np.testing.assert_array_almost_equal(scores, [1.0, 1.0, 1.0, 0.9333333333333333, 0.8, 0.8666666666666667, 1.0, 0.8666666666666667, 0.8666666666666667, 1.0])
assert np.array(scores).mean()== 0.9333333333333333

### Część 4

Tym razem w miejsce metody `KFold` użyj `StratifiedKFold`. `random_state=678` Jaki teraz otrzymujesz błąd? Czym różnią się między sobą te dwie metody?

In [114]:
"""scores = []
skf = StratifiedKFold(n_splits=10, random_state=678)

#print(skf.get_n_splits(standard_train, standard_test))

print(len(standard_train), len(standard_test), len(standard_all))

for train_index, test_index in skf.split(standard_train, train_targets):
    #print("TRAIN:", train_index, "TEST:", test_index)
    
    X_train, X_test = standard_all[train_index], standard_all[test_index]
    Y_train, Y_test = y[train_index], y[test_index]
    
    # ?
    clf = KNeighborsClassifier(n_neighbors=1, n_jobs=4).fit(X_train, Y_train)
    print(clf.score(X_test, Y_test))
    scores.append(clf.score(X_test, Y_test))

    
print ("Poszczególne wyniki: ", scores)
print ("Błąd wyznaczony procedurą walidacji krzyżowej: ", np.array(scores).mean())"""

scores = []
skf = StratifiedKFold(n_splits=10, random_state=678)

for train_index, test_index in skf.split(x, y):
    X_train, X_test = standard_all[train_index], standard_all[test_index]  # czemu to dziala???
    Y_train, Y_test = y[train_index], y[test_index]
    clf = KNeighborsClassifier(n_neighbors=1, n_jobs=4).fit(X_train, Y_train)
    print(clf.score(X_test, Y_test))
    scores.append(clf.score(X_test, Y_test))
    
print ("Poszczególne wyniki: ", scores)
print ("Błąd wyznaczony procedurą walidacji krzyżowej: ", np.array(scores).mean())

1.0
0.9333333333333333
1.0
0.9333333333333333
0.8666666666666667
1.0
0.8
1.0
1.0
1.0
Poszczególne wyniki:  [1.0, 0.9333333333333333, 1.0, 0.9333333333333333, 0.8666666666666667, 1.0, 0.8, 1.0, 1.0, 1.0]
Błąd wyznaczony procedurą walidacji krzyżowej:  0.9533333333333334


In [115]:
np.testing.assert_array_almost_equal(scores, [1.0, 0.9333333333333333, 1.0, 0.9333333333333333, 0.8666666666666667, 1.0, 0.8, 1.0, 1.0, 1.0])
assert np.array(scores).mean()== 0.9533333333333334

### Część 5

Z wykorzystaniem walidacji krzyżowej znajdź optymalny parametr `k` dla $k \in (1, 20)$. Wykorzystaj metodę przeszukującą podany zakres parametrów `GridSearchCV`.

In [1]:
clf = KNeighborsClassifier(1, n_jobs=1)
clf.get_params()

NameError: name 'KNeighborsClassifier' is not defined

In [55]:
%%time
k = np.arange(20)+1
tuned_parameters = {'n_neighbors': k}

clf = GridSearchCV(KNeighborsClassifier(n_jobs=4), tuned_parameters, cv=5).fit(standard_train, train_targets)

print(tuned_parameters)

{'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20])}
CPU times: user 632 ms, sys: 39.4 ms, total: 671 ms
Wall time: 671 ms


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [56]:
print("Zestaw najlepszych parametrów:")
print()
print(clf.best_params_)
print()
print ("Najlepszy wynik: ", clf.best_score_)
print()
print("Wyniki")
print()
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f (+/-%0.03f) dla %r"
            % (mean, std * 2, params))
print()
clf.best_estimator_

Zestaw najlepszych parametrów:

{'n_neighbors': 6}

Najlepszy wynik:  0.9523809523809523

Wyniki

0.924 (+/-0.148) dla {'n_neighbors': 1}
0.895 (+/-0.204) dla {'n_neighbors': 2}
0.943 (+/-0.135) dla {'n_neighbors': 3}
0.943 (+/-0.090) dla {'n_neighbors': 4}
0.933 (+/-0.111) dla {'n_neighbors': 5}
0.952 (+/-0.085) dla {'n_neighbors': 6}
0.943 (+/-0.092) dla {'n_neighbors': 7}
0.924 (+/-0.138) dla {'n_neighbors': 8}
0.943 (+/-0.107) dla {'n_neighbors': 9}
0.924 (+/-0.124) dla {'n_neighbors': 10}
0.933 (+/-0.093) dla {'n_neighbors': 11}
0.924 (+/-0.124) dla {'n_neighbors': 12}
0.943 (+/-0.035) dla {'n_neighbors': 13}
0.943 (+/-0.089) dla {'n_neighbors': 14}
0.933 (+/-0.072) dla {'n_neighbors': 15}
0.952 (+/-0.057) dla {'n_neighbors': 16}
0.943 (+/-0.035) dla {'n_neighbors': 17}
0.943 (+/-0.035) dla {'n_neighbors': 18}
0.933 (+/-0.050) dla {'n_neighbors': 19}
0.924 (+/-0.076) dla {'n_neighbors': 20}



KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=4, n_neighbors=6, p=2,
                     weights='uniform')

In [54]:
clf.best_estimator_.score(standard_test, test_targets)

0.9777777777777777